In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from src import style
style.set_plotly_defaults()

In [2]:
csv_path = "data/indices_fx_aligned_2005_to_today.csv"
df = pd.read_csv(csv_path, parse_dates=["Date"]).set_index("Date").sort_index()
df

,DJIA_USD,CAC40_EUR,FTSE100_GBP,NIKKEI225_JPY,USD_per_USD,USD_per_GBP,USD_per_EUR,USD_per_JPY,MXN_per_USD,MXN_per_GBP,MXN_per_EUR,MXN_per_JPY
Date,,,,,,,,,,,,
2005-01-04,10630.78027,3863.300049,4847.000000,11517.75000,1,1.883594,1.328198,0.009584,11.347000,21.373140,15.071058,0.108750
2005-01-05,10597.83008,3829.360107,4806.000000,11437.51953,1,1.885512,1.328004,0.009622,11.330000,21.362848,15.046281,0.109016
2005-01-06,10622.87988,3856.479980,4824.299805,11492.25977,1,1.876490,1.318305,0.009534,11.358000,21.313168,14.973304,0.108285
2005-01-07,10603.95996,3877.959961,4854.100098,11433.24023,1,1.871293,1.306097,0.009534,11.246000,21.044557,14.688365,0.107217
2005-01-11,10556.21973,3848.989990,4818.700195,11539.99023,1,1.878605,1.311699,0.009669,11.177000,20.997163,14.660860,0.108074
...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-09,49504.07031,8362.089844,10124.599610,51939.89063,1,1.343801,1.165787,0.006374,17.962299,24.137748,20.940206,0.114497
2026-01-13,49191.98828,8347.200195,10137.400390,53549.16016,1,1.346566,1.166698,0.006330,17.909100,24.115777,20.894508,0.113359
2026-01-14,49149.62891,8330.969727,10184.400390,54341.23047,1,1.342678,1.164253,0.006282,17.823601,23.931362,20.751176,0.111972


In [3]:
df_usd = df.copy()
df_usd["FTSE100_USD"]   = df_usd["FTSE100_GBP"]   * df_usd["USD_per_GBP"]
df_usd["CAC40_USD"]     = df_usd["CAC40_EUR"]     * df_usd["USD_per_EUR"]
df_usd["NIKKEI225_USD"] = df_usd["NIKKEI225_JPY"] * df_usd["USD_per_JPY"]

usd_cols = ["DJIA_USD", "FTSE100_USD", "CAC40_USD", "NIKKEI225_USD"]
df_usd[usd_cols]

,DJIA_USD,FTSE100_USD,CAC40_USD,NIKKEI225_USD
Date,,,,
2005-01-04,10630.78027,9129.779715,5131.225847,110.386715
2005-01-05,10597.83008,9061.769499,5085.404180,110.050223
2005-01-06,10622.87988,9052.748026,5084.015521,109.564872
2005-01-07,10603.95996,9083.441550,5064.991065,109.002191
2005-01-11,10556.21973,9052.432021,5048.716447,111.583743
...,...,...,...,...
2026-01-09,49504.07031,13605.442471,9748.411492,331.080393
2026-01-13,49191.98828,13650.674679,9738.660603,338.948780
2026-01-14,49149.62891,13674.373489,9699.354482,341.384434


In [4]:
w = pd.Series(
    {"DJIA_USD": 0.4, "FTSE100_USD": 0.3, "CAC40_USD": 0.1, "NIKKEI225_USD": 0.2},
    name="weight",
)

# sanity checks
if not np.isclose(w.sum(), 1.0):
    raise ValueError(f"Weights must sum to 1. Current sum={w.sum()}")

port_ret = (df_usd[usd_cols] * w).sum(axis=1).rename("port_ret")

df_port = pd.DataFrame({"port_ret": port_ret})
df_port

,port_ret
Date,
2005-01-04,7526.445950
2005-01-05,7488.213344
2005-01-06,7495.290886
2005-01-07,7494.915994
2005-01-11,7465.405892
...,...
2026-01-09,24924.318093
2026-01-13,24813.653532
2026-01-14,24800.375946


In [5]:
import numpy as np
import pandas as pd


# ----------------------------
# Paso (5): μ, σ^2, σ con varianza poblacional
# ----------------------------
def estimate_mu_sigma_daily(log_returns_window: np.ndarray) -> tuple[float, float, float]:
    """
    Recibe r (log-retornos diarios) y devuelve:
      μ = mean(r)
      σ^2 = var(r) poblacional (ddof=0)
      σ = sqrt(σ^2)
    """

    return mu, sig2, sig


# ----------------------------
# Paso (6): fórmula literal para simular S_T
# ----------------------------
def simulate_price_T_from_formula6(
    S_t: float,
    mu: float,
    sig2: float,
    sig: float,
    horizon_days: int,
    n_sims: int,
    rng: np.random.Generator
) -> np.ndarray:
    """
    Fórmula literal del punto 6:
      S_T = S_t * exp( (mu - sig^2/2)*(T-t) + sig*sqrt(T-t)*eps )
    donde eps ~ N(0,1) representa el cuantil.
    Aquí (T-t) = horizon_days (en días).
    """
    eps = rng.standard_normal(n_sims)  # un vector: 1 shock por escenario (1 activo)
    exponent = (mu - 0.5 * sig2) * horizon_days + sig * np.sqrt(horizon_days) * eps
    return S_t * np.exp(exponent)


# ----------------------------
# Pasos (7)-(8): valor del portafolio (1 activo) y P/G
# ----------------------------
def pnl_from_prices(S_t: float, S_T: np.ndarray, units: float) -> tuple[float, np.ndarray]:
    """
    Para 1 activo, el "portafolio" es:
      π_t = units * S_t
      π_T = units * S_T
      P/G = π_T - π_t
    units = número de títulos (acciones, ETFs, etc.).
    """
    pi_t = float(units * S_t)
    pi_T = units * S_T
    pnl = pi_T - pi_t
    return pi_t, pnl


# ----------------------------
# Pasos (9)-(11): VaR y CVaR desde P/G ordenado
# ----------------------------
def var_cvar_from_pnl(pnl: np.ndarray, confidence: float) -> tuple[float, float, int]:
    """
    Paso 10:
      VaR = observación correspondiente a (1-α)% * n, con α = confidence.
    Reportamos VaR y CVaR como pérdidas POSITIVAS.
    """
    alpha = float(confidence)
    tail = 1.0 - alpha
    pnl_sorted = np.sort(pnl)  # menor a mayor (pérdidas grandes al inicio)

    n = pnl_sorted.size
    k = max(int(np.ceil(tail * n)), 1)          # #obs en la cola
    q = float(pnl_sorted[k - 1])               # umbral cola izquierda

    VaR = max(0.0, -q)
    CVaR = max(0.0, -float(np.mean(pnl_sorted[:k])))
    return VaR, CVaR, k


# ----------------------------
# Main: rolling VaR/CVaR según pasos 1–13 (1 activo)
# ----------------------------
def montecarlo_var_from_prices_single_asset(
    prices: pd.Series,
    units: float = 1.0,
    confidence: float = 0.99,      # α (confianza)
    window_prices: int = 501,      # j=0..500 -> 501 precios -> 500 log-retornos
    horizon_days: int = 1,         # (T-t) en el punto 6 (puede ser 1 o, p.ej., 252)
    n_sims: int = 10_000,
    var_days_for_scaling: int = 10, # paso 12 (si quieres VaR_10 = VaR_1*sqrt(10))
    capital_k: float = 3.0,         # paso 13
    seed: int | None = 123
) -> pd.DataFrame:
    """
      (1) S_{t} precio hoy
      (2) ventana de 501 precios
      (3) se asume ya están en moneda reportada
      (4) log-retornos
      (5) μ, σ^2, σ (varianza poblacional)
      (6) simular S_T con fórmula literal (T-t = horizon_days)
      (7) π_T = units * S_T
      (8) P/G = π_T - π_t
      (9)-(11) VaR y CVaR
      (12) VaR_T = VaR_1 * sqrt(T) (opcional; aquí lo calculo si horizon_days==1)
      (13) Capital = k * VaR_T

    """
    rng = np.random.default_rng(seed)

    # (2)-(4)
    prices = prices.sort_index().astype(float)
    lr = np.log(prices / prices.shift(1)).dropna()

    var_list, cvar_list, port_t_list, dates = [], [], [], []

    # para paso (12)-(13) con VaR_1d: solo tiene sentido si horizon_days == 1
    var1_list = []

    for i in range(len(lr)):
        date_t = lr.index[i]

        # necesitamos 501 precios => 500 log-retornos en ventana
        # lr tiene 1 menos que precios, así que revisamos con precios directamente
        idx_price_t = prices.index.get_loc(date_t)  # date_t existe en prices
        if idx_price_t + 1 < window_prices:
            var_list.append(np.nan)
            cvar_list.append(np.nan)
            port_t_list.append(np.nan)
            var1_list.append(np.nan)
            dates.append(date_t)
            continue

        window_slice = prices.iloc[idx_price_t + 1 - window_prices : idx_price_t + 1]
        S_t = float(window_slice.iloc[-1])

        # (4) log-retornos de esa ventana (500 obs)
        r = np.diff(np.log(window_slice.to_numpy()))

        # (5) μ, σ^2, σ (poblacional)
        mu = float(np.mean(r))
        sig2 = float(np.var(r, ddof=0))  # varianza poblacional
        sig = float(np.sqrt(sig2))

        # (6) simular S_T a horizonte horizon_days
        S_T = simulate_price_T_from_formula6(
            S_t=S_t, mu=mu, sig2=sig2, sig=sig,
            horizon_days=1, n_sims=n_sims, rng=rng
        )

        # (7)-(8) P/G
        pi_t, pnl = pnl_from_prices(S_t=S_t, S_T=S_T, units=units)

        # (9)-(11) VaR/CVaR
        VaR, CVaR, k_tail = var_cvar_from_pnl(pnl, confidence=confidence)

        var_list.append(VaR)
        cvar_list.append(CVaR)
        port_t_list.append(pi_t)
        dates.append(date_t)

        # extra: VaR 1 día (para paso 12/13) si quieres mantener la lógica de las diapositivas
        if horizon_days == 1:
            var1_list.append(VaR)
        else:
            var1_list.append(np.nan)

    conf_pct = int(round(confidence * 100))
    df = pd.DataFrame(
        {
            "price": prices.reindex(dates),
            "portfolio_value_t": pd.Series(port_t_list, index=dates),
            f"VaR {conf_pct}% {horizon_days}d": pd.Series(var_list, index=dates),
            f"CVaR {conf_pct}% {horizon_days}d": pd.Series(cvar_list, index=dates),
        }
    ).dropna()

    # (12)-(13) solo si horizon_days==1 (diapositivas: primero VaR 1 día, luego escala)
    if horizon_days == 1 and var_days_for_scaling is not None:
        var_1d = df[f"VaR {conf_pct}% 1d"]
        var_T = var_1d * np.sqrt(var_days_for_scaling)
        df[f"VaR {conf_pct}% {var_days_for_scaling}d (sqrt-rule)"] = var_T
        df[f"Capital (k={capital_k}) @ {conf_pct}% {var_days_for_scaling}d"] = capital_k * var_T

    # Mensaje “paso 10”
    tail = 1.0 - confidence
    k_tail = max(int(np.ceil(tail * n_sims)), 1)
    print(
        f"MC (1 activo) | window_prices={window_prices} (≈2 años), sims={n_sims}, "
        f"conf={conf_pct}%, cola=(1-conf) -> k={k_tail} obs, "
        f"horizonte Δ={horizon_days} días en la fórmula del punto 6 (T-t=Δ)."
    )

    return df


In [6]:
df_risk = montecarlo_var_from_prices_single_asset(df_port["port_ret"], horizon_days=252)

MC (1 activo) | window_prices=501 (≈2 años), sims=10000, conf=99%, cola=(1-conf) -> k=101 obs, horizonte Δ=252 días en la fórmula del punto 6 (T-t=Δ).


In [7]:
df_risk

,price,portfolio_value_t,VaR 99% 252d,CVaR 99% 252d
2007-02-22,9598.479565,9598.479565,130.614626,149.914123
2007-02-23,9611.106027,9611.106027,130.722703,150.261919
2007-02-26,9634.550250,9634.550250,128.735074,150.291267
2007-02-27,9357.978367,9357.978367,132.254487,150.523999
2007-02-28,9303.138479,9303.138479,127.858449,147.849470
...,...,...,...,...
2026-01-09,24924.318093,24924.318093,438.162459,501.495305
2026-01-13,24813.653532,24813.653532,422.768386,484.462151
2026-01-14,24800.375946,24800.375946,435.728720,505.520805
2026-01-15,24942.613688,24942.613688,431.104528,508.978866


In [8]:
df_risk

,price,portfolio_value_t,VaR 99% 252d,CVaR 99% 252d
2007-02-22,9598.479565,9598.479565,130.614626,149.914123
2007-02-23,9611.106027,9611.106027,130.722703,150.261919
2007-02-26,9634.550250,9634.550250,128.735074,150.291267
2007-02-27,9357.978367,9357.978367,132.254487,150.523999
2007-02-28,9303.138479,9303.138479,127.858449,147.849470
...,...,...,...,...
2026-01-09,24924.318093,24924.318093,438.162459,501.495305
2026-01-13,24813.653532,24813.653532,422.768386,484.462151
2026-01-14,24800.375946,24800.375946,435.728720,505.520805
2026-01-15,24942.613688,24942.613688,431.104528,508.978866


In [9]:
targets = [(2008, 6), (2012, 12), (2014, 6), (2016, 6), (2018, 6), (2022, 6), (2025, 8), (2026, 1)]

month_ends = [
    (pd.Timestamp(y, m, 1) + pd.offsets.MonthEnd(0))
    for (y, m) in targets]

sel_dates = [df_risk.index.asof(d) for d in month_ends]

sel_dates = [d for d in sel_dates if pd.notna(d)]
seen = set()
sel_dates = [d for d in sel_dates if not (d in seen or seen.add(d))]

df_close = df_risk.loc[sel_dates].copy()
df_close.index.name = "Date"

df_close.transpose()

Date,2008-06-30,2012-12-28,2014-06-30,2016-06-30,2018-06-29,2022-06-30,2025-08-29,2026-01-16
price,8627.600956,8543.789787,10810.278549,10301.430096,13360.103774,15576.849992,22900.017700,24879.386699
portfolio_value_t,8627.600956,8543.789787,10810.278549,10301.430096,13360.103774,15576.849992,22900.017700,24879.386699
VaR 99% 252d,191.565022,220.008394,178.803042,228.041890,209.413006,375.646649,392.029327,435.926304
CVaR 99% 252d,217.881301,250.395230,201.272173,261.684847,247.311137,419.890003,452.521542,513.374935


In [10]:
df_risk

,price,portfolio_value_t,VaR 99% 252d,CVaR 99% 252d
2007-02-22,9598.479565,9598.479565,130.614626,149.914123
2007-02-23,9611.106027,9611.106027,130.722703,150.261919
2007-02-26,9634.550250,9634.550250,128.735074,150.291267
2007-02-27,9357.978367,9357.978367,132.254487,150.523999
2007-02-28,9303.138479,9303.138479,127.858449,147.849470
...,...,...,...,...
2026-01-09,24924.318093,24924.318093,438.162459,501.495305
2026-01-13,24813.653532,24813.653532,422.768386,484.462151
2026-01-14,24800.375946,24800.375946,435.728720,505.520805
2026-01-15,24942.613688,24942.613688,431.104528,508.978866


# Visualizaciones

In [11]:
r = df_risk["price"].astype(float).dropna()

fig = px.line(
    r,
    x=r.index,
    y=r.values,
    title="Evolución del valor de $1 invertido en el portafolio",
    labels={"x": "Date", "y": "Value"},
)
fig.show()

In [14]:


fig = px.line(df_risk)


fig.show()

In [15]:
df_risk

,price,portfolio_value_t,VaR 99% 252d,CVaR 99% 252d
2007-02-22,9598.479565,9598.479565,130.614626,149.914123
2007-02-23,9611.106027,9611.106027,130.722703,150.261919
2007-02-26,9634.550250,9634.550250,128.735074,150.291267
2007-02-27,9357.978367,9357.978367,132.254487,150.523999
2007-02-28,9303.138479,9303.138479,127.858449,147.849470
...,...,...,...,...
2026-01-09,24924.318093,24924.318093,438.162459,501.495305
2026-01-13,24813.653532,24813.653532,422.768386,484.462151
2026-01-14,24800.375946,24800.375946,435.728720,505.520805
2026-01-15,24942.613688,24942.613688,431.104528,508.978866
